# Project CHAN v1.0

Project Chan is an interactive Chatbot that can answer few queries. Now, lets get started. 
import libraries that we will be using in this project

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import datetime
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.chat.util import Chat, reflections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import re
warnings.filterwarnings('ignore')

# Reading the corpus data

In [2]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

# Tokenisation

In [3]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words
len(sent_tokens)

402

In [4]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


# User Greeting message

In [5]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
General_questions = {'.* your name.*':'My name is CHAN and I am a chatbot',
                    '.* your age.*':'I was brought online 2 days ago, so my age is 2 days',
                    '.* created you?':'I was brought to this world by Charan and Anubrata',
                    '.* today date':'todays date and time is '+str(datetime.datetime.now()),
                    'tell me about yourself?|what can you do?':'Well, I am a chatbot i have some idea about animals, art, countries, scientist. You can also teach me i will learn'}
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

def general_qn(sentence): 
    
    for k, v in General_questions.items():      
            if re.match(k,sentence.lower()):
                return v

# generate a response

In [6]:
def response(user_response):
    f= open("out.txt","a+")
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    f.writelines("tfidf[-1]"+str(tfidf[-1]))
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    f.writelines("tfidf"+str(tfidf))
    f.writelines("\nvals"+str(vals))
    f.writelines("\nidx"+str(idx))
    f.writelines("\nflat"+str(flat.sort()))
    f.writelines("\nreq"+str(req_tfidf))
    f.close()
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response



# Teach the chatbot new data

In [7]:
def teach_Chatbot(user_response):
    f= open("chatbot.txt","a+")
    f.writelines(user_response)
    f.close()
    sent_tokens.append(user_response)
    print('Thank you I have learned what you taught to me')    

In [8]:
flag=True
print("Welcome lets get started")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            print("CHAN: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("CHAN: "+greeting(user_response))
            elif(general_qn(user_response)!=None):
                print("CHAN: "+general_qn(user_response))
            elif(user_response=='teach'):
                teach_mode=input()
                teach_Chatbot(teach_mode)
            else:
                print("CHAN: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("CHAN: Bye! take care..")

Welcome lets get started
who is your co-founder?
CHAN: charan is my co-founder.
who is anubrata?
CHAN: anubrata is my co-founder he is a god in bml but he is an asshole.
bye
CHAN: Bye! take care..
